In [1]:
import os
import streamlit as st
import pickle
import time 
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import google.generativeai as genai
from dotenv import load_dotenv

C:\Users\Ilham\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading API KEY from .env file

In [27]:
load_dotenv()

# Access the Cohere API key (assuming you stored your Cohere key in the .env file)
gemini_api_key = os.getenv('Gemini_API_Key')
genai.configure(api_key=gemini_api_key)
os.environ['GOOGLE_API_KEY'] = gemini_api_key

### Text Splitter

In [3]:
loaders=UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/jio-financial-services-blackrock-form-joint-venture-for-investment-advisory-services-12817133.html",
    "https://www.moneycontrol.com/news/business/spicejet-chairman-ajay-singh-may-sell-10-15-stake-in-airline-to-raise-funds-12817037.html"
])

data=loaders.load()
len(data)

2

In [4]:
data[1].metadata

{'source': 'https://www.moneycontrol.com/news/business/spicejet-chairman-ajay-singh-may-sell-10-15-stake-in-airline-to-raise-funds-12817037.html'}

In [4]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs=text_splitter.split_documents(data)
len(docs)

14

In [5]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/jio-financial-services-blackrock-form-joint-venture-for-investment-advisory-services-12817133.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nFREE Credit Score₹100 Cash Reward\n\nFixed Deposits\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_BUSINESS_AS/MC_ENG_ROS_NWS_BUS_AS_ATF_728\n\nGo PRO @₹99 PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessJio Financial Services, BlackRock form joint venture for investment a

### Creating text embedding

In [31]:

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gemini_api_key)

vectorindex_gemini = FAISS.from_documents(docs, embedding=embeddings)
vectorindex_gemini

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=gemini_api_key
)


In [33]:
qa_chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_gemini.as_retriever())

## Query Testing

In [34]:
import langchain
query='How much is Jio Financial Services planning to invest in the initial subscription of 30,00,000 equity shares, and what is the face value of each share?'

langchain.debug=True

result=qa_chain({"question":query},return_only_outputs=True)
result

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "How much is Jio Financial Services planning to invest in the initial subscription of 30,00,000 equity shares, and what is the face value of each share?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Jio Financial Services in its stock exchange filing disclosed that the company will invest Rs 3 crore towards the initial subscription of 30,00,000 equity shares, each with a face value of Rs 10.\n\nStory continues below Advertisement\n\nRemove Ad\n\nDetails of the business plan are yet to be announced.\n\nThe company further stated that the Certificate of Incorporation was issued by the Ministry of Corporate Affairs on September 7, 2024.\n\nJio F

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


[llm/error] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] [11.49s] LLM run errored with error:
"ResourceExhausted('Resource has been exhausted (e.g. check quota).')Traceback (most recent call last):\n\n\n  File \"C:\\Users\\Ilham\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\langchain_core\\language_models\\chat_models.py\", line 624, in generate\n    self._generate_with_cache(\n\n\n  File \"C:\\Users\\Ilham\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\langchain_core\\language_models\\chat_models.py\", line 846, in _generate_with_cache\n    result = self._generate(\n             ^^^^^^^^^^^^^^^\n\n\n  File \"C:\\Users\\Ilham\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\langchain_google_genai\\chat_models.py\", line 975, in _generate\n    response: GenerateContentResponse = _chat_with_retry(\n                                        ^^^^^^^^^^^^^^^^^\n\n\n  Fi

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).